<a href="https://colab.research.google.com/github/katarinagresova/M7DataSP_2020/blob/main/assignment_05/Transfer_Learning_GoT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
import keras

# Data

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
pip install split-folders

This part is for creating correct structure of folders. I copied data from Petr to my Drive to location `/content/drive/My Drive/M7DataSP/GoT_data/`. Function `splitfolders.ratio()` creates `train` folder with 80% of data and `val` folder with remaining 20%. In both folders there are three subfolders coresponding to each GoT character.

In [ ]:
import splitfolders
splitfolders.ratio('/content/drive/My Drive/M7DataSP/GoT_data/', output="/content/drive/My Drive/M7DataSP/GoT_data/", seed=1337, ratio=(.8, .2))

In [6]:
PATH = '/content/drive/My Drive/M7DataSP/GoT_data/'

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'val')

# MobileNetV2

In [7]:
BATCH_SIZE = 32
EPOCHS = 10
IMG_SIZE = (160, 160)
IMG_SHAPE = IMG_SIZE + (3,)

In [8]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
________________

# Transfer Learning

## Model

In [9]:
for layer in base_model.layers:
    layer.trainable = False
    
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(units=3, activation="softmax")(x)

model = tf.keras.Model(inputs=base_model.input, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Data Generators

In [10]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

Found 661 images belonging to 3 classes.
Found 170 images belonging to 3 classes.


## Training

In [13]:
history = model.fit(train_generator,
                    epochs=EPOCHS,
                    validation_data=validation_generator)

Epoch 1/10
21/21 [==============================] - ETA: 0s - loss: 1.0861 - accuracy: 0.5129

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


21/21 [==============================] - 27s 1s/step - loss: 1.0861 - accuracy: 0.5129 - val_loss: 0.6850 - val_accuracy: 0.7235
Epoch 2/10
21/21 [==============================] - 25s 1s/step - loss: 0.7070 - accuracy: 0.6944 - val_loss: 0.5612 - val_accuracy: 0.7706
Epoch 3/10
21/21 [==============================] - 25s 1s/step - loss: 0.5735 - accuracy: 0.7549 - val_loss: 0.5010 - val_accuracy: 0.7941
Epoch 4/10
21/21 [==============================] - 25s 1s/step - loss: 0.5555 - accuracy: 0.7761 - val_loss: 0.4953 - val_accuracy: 0.7882
Epoch 5/10
21/21 [==============================] - 25s 1s/step - loss: 0.4829 - accuracy: 0.8321 - val_loss: 0.4707 - val_accuracy: 0.7941
Epoch 6/10
21/21 [==============================] - 25s 1s/step - loss: 0.4141 - accuracy: 0.8427 - val_loss: 0.4572 - val_accuracy: 0.8059
Epoch 7/10
21/21 [==============================] - 25s 1s/step - loss: 0.4055 - accuracy: 0.8487 - val_loss: 0.4486 - val_accuracy: 0.8176
Epoch 8/10
21/21 [=============

# Export to TFjs

In [13]:
!pip install tensorflowjs

     |████████████████████████████████| 71kB 3.7MB/s 
     |████████████████████████████████| 112kB 15.2MB/s 
  Found existing installation: tensorflow-hub 0.10.0
    Uninstalling tensorflow-hub-0.10.0:
      Successfully uninstalled tensorflow-hub-0.10.0


In [14]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, PATH + "/export_model")

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)
